# Debug missing prate 

Parquet datasets are missing a prate column and I want to figure out why

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dask
import dask_jobqueue
import dask.dataframe as dd
import dask.distributed
import os
import pathlib
import pandas as pd

import xarray as xr

In [ ]:
from smc01.interpolate.extractor import DEFAULT_EXTRACTOR, ShortNameLevelExtractor
from smc01.interpolate.forecast_reader import do_one_file
from smc01.interpolate.interpolate import pass_to_xarray

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))
PASS_DIR = DATA_DIR / 'data/gdps/2021062700'
pass_path = pathlib.Path(PASS_DIR)

In [ ]:
pass_files = sorted([f for f in pass_path.iterdir()])

In [ ]:
pass_files[:10]

In [ ]:
_, first_dataset = do_one_file(pass_files[0], DEFAULT_EXTRACTOR)

In [ ]:
_, last_dataset = do_one_file(pass_files[-1], DEFAULT_EXTRACTOR)

In [ ]:
first_dataset

In [ ]:
last_dataset

In [ ]:
set(first_dataset.data_vars) ^ set(last_dataset.data_vars)

In [ ]:
gh_extractor = ShortNameLevelExtractor('gh', [1000])

In [ ]:
pass_files[-1]

In [ ]:
for f in pass_files[-3:]:
    _, dataset = do_one_file(f, gh_extractor)
    print(f)
    print(dataset.data_vars)

## Prate is missing

In [ ]:
test_file = DATA_DIR / 'interpolated/2021-12-20-test/2021100500.parquet'

In [ ]:
df = pd.read_parquet(test_file)

In [ ]:
df.columns

In [ ]:
df['gdps_prate']

In [ ]:
df[df['step'] == 0]

In [ ]:
test_dataset = DATA_DIR / 'interpolated/2021-11-30-interpolated/'

prate is actually present for some of the forecasts. I will make a more precise count of where prate is present and where it is absent.

### Count missing prates

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ~/.bash_profile','conda activate smc01'],
    name='smc01-dask',
)

In [ ]:
cluster.scale(jobs=4)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

In [ ]:
dataset_files = sorted(list(pathlib.Path(DATA_DIR / 'interpolated/2021-11-30-interpolated/').glob('*.parquet')))

It's missing in one of the files but most of them seem to have it

In [ ]:
no_prate = set()
for f in dataset_files[-10:]:
    df = pd.read_parquet(f)
    if 'gdps_prate' not in df.columns:
        no_prate.add(f)

In [ ]:
no_prate

In [ ]:
import pygrib
from smc01.interpolate.extractor import DEFAULT_EXTRACTOR

In [ ]:
sample_file = DATA_DIR / 'data/gdps/2020120712/CMC_glb_latlon.24x.24_2020120712_P000.grib2'

In [ ]:
grib_file = pygrib.open(sample_file)

In [ ]:
for m in grib_file:
    print(DEFAULT_EXTRACTOR(m))

prate goes through the extractor but seems to be lost afterwards

In [ ]:
from smc01.interpolate.forecast_reader import do_one_file

In [ ]:
filename, dataset = do_one_file(sample_file, DEFAULT_EXTRACTOR)

In [ ]:
dataset

In [ ]:
dataset.prate.plot()

I bet the 2020 data has prate in the 0th step and not 2021.
I will test a concatenation with 2021 files, one 0th step and one first step.

In [ ]:
_, f1 = do_one_file(DATA_DIR / 'data/gdps/2021120712/CMC_glb_2021120712_P000.grib2', DEFAULT_EXTRACTOR)
_, f2 = do_one_file(DATA_DIR / 'data/gdps/2021120712/CMC_glb_2021120712_P003.grib2', DEFAULT_EXTRACTOR)

In [ ]:
merged = xr.merge([f1, f2])

In [ ]:
merged['step']

In [ ]:
merged.isel(step=1).prate.plot()

In [ ]:
f1

In [ ]:
f2